In [1]:
import os
import datetime as dt

import pathlib

import pandas as pd

import rasterio
import geopandas as gpd

In [2]:
# Get list of paths
data_dir = '/Users/patmccornack/Documents/ucsb_fog_project/data/gis/pisr_ts_rasters'
paths = list(pathlib.Path(data_dir).glob('*.tif'))
paths.sort()
print(paths)

[PosixPath('/Users/patmccornack/Documents/ucsb_fog_project/data/gis/pisr_ts_rasters/rad_2023-03-01_10.0.tif'), PosixPath('/Users/patmccornack/Documents/ucsb_fog_project/data/gis/pisr_ts_rasters/rad_2023-03-01_10.5.tif'), PosixPath('/Users/patmccornack/Documents/ucsb_fog_project/data/gis/pisr_ts_rasters/rad_2023-03-01_11.0.tif'), PosixPath('/Users/patmccornack/Documents/ucsb_fog_project/data/gis/pisr_ts_rasters/rad_2023-03-01_11.5.tif'), PosixPath('/Users/patmccornack/Documents/ucsb_fog_project/data/gis/pisr_ts_rasters/rad_2023-03-01_12.0.tif'), PosixPath('/Users/patmccornack/Documents/ucsb_fog_project/data/gis/pisr_ts_rasters/rad_2023-03-01_12.5.tif'), PosixPath('/Users/patmccornack/Documents/ucsb_fog_project/data/gis/pisr_ts_rasters/rad_2023-03-01_13.0.tif'), PosixPath('/Users/patmccornack/Documents/ucsb_fog_project/data/gis/pisr_ts_rasters/rad_2023-03-01_13.5.tif'), PosixPath('/Users/patmccornack/Documents/ucsb_fog_project/data/gis/pisr_ts_rasters/rad_2023-03-01_14.0.tif'), PosixPath

In [3]:
# Create DF to store results
results_df = pd.DataFrame(columns=['datetime', 'potential_solar_rad'])

# Open points file and get station coords
pointData = gpd.read_file('/Users/patmccornack/Documents/ucsb_fog_project/data/gis/points/station_pts.shp')
station_pt = pointData.iloc[0]['geometry']
x = station_pt.xy[0][0]
y = station_pt.xy[1][0]

# Extract values
for path in paths: 
    # Open raster file
    raster = rasterio.open(path)

    # Get hour from fname
    fname = path.stem
    date = fname.split('_')[-2]
    hour = fname.split('_')[-1]
    hour_dt = str(dt.timedelta(hours=float(hour)))

    time = dt.datetime.strptime(f'{date} {hour_dt}', '%Y-%m-%d %H:%M:%S')

    #hour = fname.split('_')[-1]
    #hour_dt = str(dt.timedelta(hours=float(hour)))

    # Extract point value from raster
    row, col = raster.index(x,y)
    raster_value = raster.read(1)[row,col]

    results_df.loc[len(results_df)] = [time, raster_value]

months = [datetime.month for datetime in results_df['datetime']]
days = [datetime.day for datetime in results_df['datetime']]
times = [str(datetime.time()) for datetime in results_df['datetime']]

results_df['month'] = months
results_df['day'] = days
results_df['clock_time'] = times

results_df.sort_values(by='datetime', inplace=True)
results_df.head()


,datetime,potential_solar_rad,month,day,clock_time
16,2023-03-01 07:00:00,32.467213,3,1,07:00:00
17,2023-03-01 07:30:00,58.696186,3,1,07:30:00
18,2023-03-01 08:00:00,93.152328,3,1,08:00:00
19,2023-03-01 08:30:00,159.439026,3,1,08:30:00
20,2023-03-01 09:00:00,236.069427,3,1,09:00:00


In [4]:
out_dir = "/Users/patmccornack/Documents/ucsb_fog_project/data/csv"
results_df.to_csv(os.path.join(out_dir, 'potential_solar_rad.csv'))
